In [1]:
from nltk.corpus import gutenberg # to get bible corpus
from string import punctuation # to remove punctuation from corpus
import nltk 
import re
import numpy as np
import pandas as pd
from keras.preprocessing import text
from keras.preprocessing.sequence import skipgrams 
from keras.layers import *
from keras.layers.core import Dense, Reshape
from keras.layers.embeddings import Embedding
from keras.models import Model,Sequential

In [2]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

In [3]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [4]:
verse = pd.read_csv('./bible/t_asv.csv', index_col='id', 
                    dtype={'id': np.int64, 'b': np.int32, 'c': np.int32, 'v': np.int32, 't': object})

verse.head()


,b,c,v,t
id,,,,
1001001,1,1,1,In the beginning God created the heavens and t...
1001002,1,1,2,And the earth was waste and void; and darkness...
1001003,1,1,3,"And God said, Let there be light: and there wa..."
1001004,1,1,4,"And God saw the light, that it was good: and G..."
1001005,1,1,5,"And God called the light Day, and the darkness..."


In [5]:
sentences = verse.t.to_list()

samples =sentences

In [6]:
corpus_raw = ' '.join(samples)


In [7]:
bible = corpus_raw.split( )

In [8]:
tokens = list(corpus_raw.lower().split())
print(len(tokens))

785109


In [9]:
vocab, index = {}, 1  # start indexing from 1
vocab['<pad>'] = 0  # add a padding token
for token in tokens:
    if token not in vocab:
        vocab[token] = index
        index += 1
vocab_size = len(vocab)
print(vocab)

{'<pad>': 0, 'in': 1, 'the': 2, 'beginning': 3, 'god': 4, 'created': 5, 'heavens': 6, 'and': 7, 'earth.': 8, 'earth': 9, 'was': 10, 'waste': 11, 'void;': 12, 'darkness': 13, 'upon': 14, 'face': 15, 'of': 16, 'deep:': 17, 'spirit': 18, 'moved': 19, 'waters.': 20, 'said,': 21, 'let': 22, 'there': 23, 'be': 24, 'light:': 25, 'light.': 26, 'saw': 27, 'light,': 28, 'that': 29, 'it': 30, 'good:': 31, 'divided': 32, 'light': 33, 'from': 34, 'darkness.': 35, 'called': 36, 'day,': 37, 'he': 38, 'night.': 39, 'evening': 40, 'morning,': 41, 'one': 42, 'day.': 43, 'a': 44, 'firmament': 45, 'midst': 46, 'waters,': 47, 'divide': 48, 'waters': 49, 'made': 50, 'firmament,': 51, 'which': 52, 'were': 53, 'under': 54, 'above': 55, 'firmament:': 56, 'so.': 57, 'heaven.': 58, 'second': 59, 'gathered': 60, 'together': 61, 'unto': 62, 'place,': 63, 'dry': 64, 'land': 65, 'appear:': 66, 'earth;': 67, 'gathering': 68, 'seas:': 69, 'good.': 70, 'put': 71, 'forth': 72, 'grass,': 73, 'herbs': 74, 'yielding': 75, 

In [10]:
inverse_vocab = {index: token for token, index in vocab.items()}


In [11]:
example_sequence = [vocab[word] for word in tokens]
print(example_sequence[:10])

[1, 2, 3, 4, 5, 2, 6, 7, 2, 8]


In [12]:
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      example_sequence,
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0)
print(len(positive_skip_grams))

3140430


In [13]:
for target, context in positive_skip_grams[:5]:
    print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(7, 3783): (and, ourselves,)
(372, 194): (ye, shall)
(1409, 234): (king, but)
(348, 276): (now, name)
(189, 194): (you, shall)


In [14]:
# Get target and context words for one positive skip-gram.
target_word, context_word = positive_skip_grams[0]

# Set the number of negative samples per positive context.
num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=vocab_size,  # pick index of the samples from [0, vocab_size]
    seed=SEED,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

tf.Tensor([ 579   40 4079  151], shape=(4,), dtype=int64)
['hid;', 'evening', 'ingatherings,', 'creatures']


In [15]:
# Add a dimension so you can use concatenation (in the next step).
negative_sampling_candidates = tf.expand_dims(negative_sampling_candidates, 1)

# Concatenate a positive context word with negative sampled words.
context = tf.concat([context_class, negative_sampling_candidates], 0)

# Label the first context word as `1` (positive) followed by `num_ns` `0`s (negative).
label = tf.constant([1] + [0]*num_ns, dtype="int64")

# Reshape the target to shape `(1,)` and context and label to `(num_ns+1,)`.
target = tf.squeeze(target_word)
context = tf.squeeze(context)
label = tf.squeeze(label)

In [16]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

target_index    : 7
target_word     : and
context_indices : [3783  579   40 4079  151]
context_words   : ['ourselves,', 'hid;', 'evening', 'ingatherings,', 'creatures']
label           : [1 0 0 0 0]


In [17]:
print("target  :", target)
print("context :", context)
print("label   :", label)

target  : tf.Tensor(7, shape=(), dtype=int32)
context : tf.Tensor([3783  579   40 4079  151], shape=(5,), dtype=int64)
label   : tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)


In [18]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)

[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


In [19]:
text_ds = tf.data.Dataset.from_tensor_slices(tokens)

In [20]:
text_ds

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [21]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [22]:
vectorize_layer.adapt(text_ds.batch(1024))

In [25]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

785109


In [26]:
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[7 0 0 0 0 0 0 0 0 0] => ['in', '', '', '', '', '', '', '', '', '']
[2 0 0 0 0 0 0 0 0 0] => ['the', '', '', '', '', '', '', '', '', '']
[738   0   0   0   0   0   0   0   0   0] => ['beginning', '', '', '', '', '', '', '', '', '']
[31  0  0  0  0  0  0  0  0  0] => ['god', '', '', '', '', '', '', '', '', '']
[1323    0    0    0    0    0    0    0    0    0] => ['created', '', '', '', '', '', '', '', '', '']


In [27]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)[:,:,0]
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

NameError: name 'generate_training_data' is not defined

In [ ]:
vectorize_layer.adapt(text_ds.batch(1024))

In [ ]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
        context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
        negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=SEED,
          name="negative_sampling")

          # Build context and label vectors (for one target word)
        negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

        context = tf.concat([context_class, negative_sampling_candidates], 0)
        label = tf.constant([1] + [0]*num_ns, dtype="int64")

        # Append each element from the training example to global lists.
        targets.append(target_word)
        contexts.append(context)
        labels.append(label)

    return targets, contexts, labels

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

In [ ]:
class Word2Vec(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2Vec, self).__init__()
        self.target_embedding = layers.Embedding(vocab_size,
                                          embedding_dim,
                                          input_length=1,
                                          name="w2v_embedding")
        self.context_embedding = layers.Embedding(vocab_size,
                                           embedding_dim,
                                           input_length=num_ns+1)

    def call(self, pair):
        target, context = pair
        # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
        # context: (batch, context)
        if len(target.shape) == 2:
            target = tf.squeeze(target, axis=1)
        # target: (batch,)
        word_emb = self.target_embedding(target)
        # word_emb: (batch, embed)
        context_emb = self.context_embedding(context)
        # context_emb: (batch, context, embed)
        dots = tf.einsum('be,bce->bc', word_emb, context_emb)
        # dots: (batch, context)
        return dots

In [ ]:
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

In [ ]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [ ]:
word2vec.fit(target, epochs=20, callbacks=[tensorboard_callback])

In [ ]:
wpt = nltk.WordPunctTokenizer()

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc,re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

In [ ]:
norm_bible = [[word.lower() for word in sent if word not in remove_terms] for sent in bible]
norm_bible = [' '.join(tok_sent) for tok_sent in norm_bible]
norm_bible = filter(None, normalize_corpus(norm_bible))
norm_bible = [tok_sent for tok_sent in norm_bible if len(tok_sent.split()) > 2]